In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib as matlib
from numpy import nan
from numpy.linalg import norm as norm
import os
import time as tmp
import commands
import copy

import tsid
import pinocchio as pin

import sys
sys.path.append('..')

import plot_utils as plut

In [2]:
import gepetto.corbaserver


## Test Task Space Inverse Dynamics

In [3]:
lxp = 0.1                           # foot length in positive x direction
lxn = 0.11                          # foot length in negative x direction
lyp = 0.069                         # foot length in positive y direction
lyn = 0.069                         # foot length in negative y direction
lz = 0.107                          # foot sole height with respect to ankle joint
mu = 0.3                            # friction coefficient
fMin = 5.0                          # minimum normal force
fMax = 1500.0                       # maximum normal force

rf_frame_name = "leg_right_6_joint"          # right foot joint name
lf_frame_name = "leg_left_6_joint"           # left foot joint name
contactNormal = np.matrix([0., 0., 1.]).T    # direction of the normal to the contact surface

w_com = 1.0                       # weight of center of mass task
w_posture = 0.75                  # weight of joint posture task
w_forceRef = 1e-3                 # weight of force regularization task
w_waist = 1.0                     # weight of waist task

kp_contact = 30.0                 # proportional gain of contact constraint
kp_com = 3000.0                   # proportional gain of center of mass task
kp_posture = 30.0                 # proportional gain of joint posture task
kp_waist = 500.0                  # proportional gain of waist task

dt = 0.001                        # controller time step
PRINT_N = 500                     # print every PRINT_N time steps
DISPLAY_N = 20                    # update robot configuration in viwewer every DISPLAY_N time steps
N_SIMULATION = 4000               # number of time steps simulated

In [4]:
path = "/opt/openrobots/share"
urdf = path + '/talos_data/urdf/talos_reduced.urdf'
vector = pin.StdVec_StdString()
vector.extend(item for item in path)
robot = tsid.RobotWrapper(urdf, vector, pin.JointModelFreeFlyer(), False)
srdf = path + '/talos_data/srdf/talos.srdf'


In [5]:
# for gepetto viewer
robot_display = pin.RobotWrapper.BuildFromURDF(urdf, [path, ], pin.JointModelFreeFlyer())
l = commands.getstatusoutput("ps aux |grep 'gepetto-gui'|grep -v 'grep'|wc -l")
if int(l[1]) == 0:
    os.system('gepetto-gui &')
tmp.sleep(1)
cl = gepetto.corbaserver.Client()
gui = cl.gui
robot_display.initDisplay(loadModel=True)


In [6]:
#help(pin.loadReferenceConfigurations)
#help(robot.model().referenceConfigurations)

In [7]:
model = robot.model()
pin.loadReferenceConfigurations(model, srdf, False)
q = model.referenceConfigurations['half_sitting']
v = np.matrix(np.zeros(robot.nv)).T


In [8]:
robot_display.displayCollisions(False)
robot_display.displayVisuals(True)
robot_display.display(q)

In [9]:
assert model.existFrame(rf_frame_name)
assert model.existFrame(lf_frame_name)

In [10]:
t = 0.0 # time
invdyn = tsid.InverseDynamicsFormulationAccForce("tsid", robot, False)
invdyn.computeProblemData(t, q, v)
data = invdyn.data()

#data_model = model.createData()
#pin.forwardKinematics(model, data_model, q)
#pin.centerOfMass(model, data_model, q)

## Test forceConstraint
________________________________________________________________________________________________________________________

### End Test
________________________________________________________________________________________________________________________

In [11]:
# COM
comTask = tsid.TaskComEquality("task-com", robot)
comTask.setKp(kp_com * matlib.ones(3).T)
comTask.setKd(2.0 * np.sqrt(kp_com) * matlib.ones(3).T)
invdyn.addMotionTask(comTask, w_com, 1, 0.0)

#print "com acc: " + str(comTask.getDesiredAcceleration)

# WAIST
waistTask = tsid.TaskSE3Equality("keepWaist", robot, 'root_joint')
kp = kp_waist * matlib.ones(6).T
waistTask.setKp(kp)
kd = 2.0 * np.sqrt(kp_waist) * matlib.ones(6).T
waistTask.setKd(kd)
mask = matlib.ones(6).T
mask[:3] = 0.
waistTask.setMask(mask)
invdyn.addMotionTask(waistTask, w_waist, 1, 0.0)
#print "waist acc: " + str(waistTask.getDesiredAcceleration)

# POSTURE
postureTask = tsid.TaskJointPosture("task-posture", robot)
postureTask.setKp(kp_posture * matlib.ones(robot.nv-6).T)
kd = 2.0 * np.sqrt(kp_posture) * matlib.ones(robot.nv-6).T
postureTask.setKd(kd)
mask = matlib.ones(robot.nv-6).T
mask[:11] = 0.
#print(mask)
postureTask.mask(mask)
#invdyn.addMotionTask(postureTask, w_posture, 1, 0.0)


# END EFFECTOR 

w_eff = 0.9 #weight of end effector task
kp_eff = 50.0                     # proportional gain of end effector task

rhJointName = "gripper_right_joint" #look up in the urdf

rhTask = tsid.TaskSE3Equality("task-endEffector", robot, rhJointName)    
rhTask.setKp(kp_eff * np.matrix(np.ones(6)).transpose())   
rhTask.setKd(2.0 * np.sqrt(kp_eff) * np.matrix(np.ones(6)).transpose()) 
#invdyn.addMotionTask(rhTask, w_eff, 1, 0.0)

#WALL
wallTask = tsid.TaskWall("task-wall", robot, rhJointName)
#print robot.nv
#print "post acc: " + str(postureTask.getDesiredAcceleration)

AttributeError: 'module' object has no attribute 'TaskWall'

In [12]:
# CONTACTS
contact_Point = np.matrix(np.ones((3,4)) * lz)
contact_Point[0, :] = [-lxn, -lxn, lxp, lxp]
contact_Point[1, :] = [-lyn, lyp, -lyn, lyp]
#print contact_Point

contactRF = tsid.Contact6d("contact_rfoot", robot, rf_frame_name, contact_Point, contactNormal, mu, fMin, fMax)
contactRF.setKp(kp_contact * matlib.ones(6).T)
contactRF.setKd(2.0 * np.sqrt(kp_contact) * matlib.ones(6).T)
H_rf_ref = robot.position(data, model.getJointId(rf_frame_name))
print(H_rf_ref)
contactRF.setReference(H_rf_ref)
invdyn.addRigidContact(contactRF, w_forceRef)

contactLF = tsid.Contact6d("contact_lfoot", robot, lf_frame_name, contact_Point, contactNormal, mu, fMin, fMax)
contactLF.setKp(kp_contact * matlib.ones(6).T)
contactLF.setKd(2.0 * np.sqrt(kp_contact) * matlib.ones(6).T)
H_lf_ref = robot.position(data, model.getJointId(lf_frame_name))
contactLF.setReference(H_lf_ref)
invdyn.addRigidContact(contactLF, w_forceRef)

  R =
        1         0         0
        0  0.999999  0.001708
        0 -0.001708  0.999999
  p = -0.00884695      -0.085    0.106998



True

In [13]:
def listObjectData(obj):
    print("Class : "+str(type(obj)))
    members = [attr for attr in dir(obj) if not callable(getattr(obj, attr)) and not attr.startswith("__")]    
    print("list of members :")
    print members
    for m in members:
        print(str(m)+":")
        print(getattr(obj, m) )
        
        
def listObjectFunctions(obj):
    print("Class : "+str(type(obj)))
    members = [attr for attr in dir(obj) if callable(getattr(obj, attr)) and not attr.startswith("__")]    
    print("list of functions :")
    print members
    for m in members:
        print(str(m)+":")
        print(getattr(obj, m) )

def listModelJoints(modelol):
    #first we get the number of joints
    nbJoints = modelol.getJointId("AAAAAAAAAAAAAH")#getJointId returns the number of joints if the joint asked does not exist
    #listObjectFunctions(model) #can be used to list all joints..?


In [14]:
com_ref = data.com[0]
#for i in data_model.com:
#    print i

print("Com ref :")
print com_ref
print("End com ref")
trajCom = tsid.TrajectoryEuclidianConstant("traj_com", com_ref)

q_ref = q[7:]
#print len(q_ref)
trajPosture = tsid.TrajectoryEuclidianConstant("traj_joint", q_ref)

waist_ref = robot.position(data, model.getJointId('root_joint'))
print("Waist ref :")
print waist_ref
print("End waist ref")
#listObjectData(waist_ref)

trajWaist = tsid.TrajectorySE3Constant("traj_waist", waist_ref)

#End effector trajectory
#Position que l'on veut donner à la main

rh_ref = copy.copy(robot.position(data, model.getJointId(rhJointName))) #on recupere la position de la main
#print("rh ref :")
#print(rh_ref)
#print("End rh ref")
#print(rh_ref.translation)

#On passe par un truc temporaire parce qu'on peut apparemment pas éditer directement rh_ref.translation
temp_translation = rh_ref.translation
temp_translation[0]+= 1 #x movement
temp_translation[1]+= 0 #y movement
temp_translation[2]+= 0 #z movement
rh_ref.translation = temp_translation
rhdestinationpos = rh_ref.translation

trajRh = tsid.TrajectorySE3Constant("traj_rh",rh_ref)

Com ref :
[[-0.0031639 ]
 [ 0.00123738]
 [ 0.87668139]]
End com ref
Waist ref :
  R =
1 0 0
0 1 0
0 0 1
  p =       0       0 1.01927

End waist ref


In [15]:
# Solver
solver = tsid.SolverHQuadProgFast("qp solver")
solver.resize(invdyn.nVar, invdyn.nEq, invdyn.nIn)

In [ ]:
com_pos = matlib.empty((3, N_SIMULATION))*nan
com_vel = matlib.empty((3, N_SIMULATION))*nan
com_acc = matlib.empty((3, N_SIMULATION))*nan

com_pos_ref = matlib.empty((3, N_SIMULATION))*nan
com_vel_ref = matlib.empty((3, N_SIMULATION))*nan
com_acc_ref = matlib.empty((3, N_SIMULATION))*nan
com_acc_des = matlib.empty((3, N_SIMULATION))*nan # acc_des = acc_ref - Kp*pos_err - Kd*vel_err

err_x = []
err_y = []
err_z = []
for i in range(0, N_SIMULATION):
    time_start = tmp.time()

    #compute samples
    sampleCom = trajCom.computeNext()
    comTask.setReference(trajCom.computeNext())
    
    samplePosture = trajPosture.computeNext()
    postureTask.setReference(samplePosture)
    
    sampleWaist = trajWaist.computeNext()
    waistTask.setReference(sampleWaist)
    
    sampleRh = trajRh.computeNext()
    rhTask.setReference(sampleRh)

    HQPData = invdyn.computeProblemData(t, q, v)
    # if i == 0: HQPData.print_all()

    sol = solver.solve(HQPData)
    if(sol.status!=0):
        print "QP problem could not be solved! Error code:", sol.status
        break
    
    tau = invdyn.getActuatorForces(sol)
    dv = invdyn.getAccelerations(sol)
    
    com_pos[:,i] = robot.com(invdyn.data())
    com_vel[:,i] = robot.com_vel(invdyn.data())
    com_acc[:,i] = comTask.getAcceleration(dv)
    com_pos_ref[:,i] = sampleCom.pos()
    com_vel_ref[:,i] = sampleCom.vel()
    com_acc_ref[:,i] = sampleCom.acc()
    com_acc_des[:,i] = comTask.getDesiredAcceleration

    if i%PRINT_N == 0:
        print "Time %.3f"%(t)
        if invdyn.checkContact(contactRF.name, sol):
            f = invdyn.getContactForce(contactRF.name, sol)
            print "\tnormal force %s: %.1f"%(contactRF.name.ljust(20,'.'),contactRF.getNormalForce(f))

        if invdyn.checkContact(contactLF.name, sol):
            f = invdyn.getContactForce(contactLF.name, sol)
            print "\tnormal force %s: %.1f"%(contactLF.name.ljust(20,'.'),contactLF.getNormalForce(f))
# 
        print "\ttracking err %s: %.3f"%(comTask.name.ljust(20,'.'),       norm(comTask.position_error, 2))
        print "\t||v||: %.3f\t ||dv||: %.3f"%(norm(v, 2), norm(dv))

    v_mean = v + 0.5*dt*dv
    v += dt*dv
    q = pin.integrate(model, q, dt*v_mean)
    t += dt
    
    if i%DISPLAY_N == 0: robot_display.display(q)

    rhpos = robot.position(data, model.getJointId(rhJointName)).translation
    err_x.append(rhTask.position_error[0,0])
    err_y.append(rhTask.position_error[1,0])
    err_z.append(rhTask.position_error[2,0])
    time_spent = tmp.time() - time_start
    if(time_spent < dt): tmp.sleep(dt-time_spent)

Time 0.000
	normal force contact_rfoot.......: 442.8
	normal force contact_lfoot.......: 442.8
	tracking err task-com............: 0.000
	||v||: 0.000	 ||dv||: 2.083
Time 0.500
	normal force contact_rfoot.......: 442.6
	normal force contact_lfoot.......: 443.0
	tracking err task-com............: 0.000
	||v||: 1.219	 ||dv||: 2.176
Time 1.000
	normal force contact_rfoot.......: 442.8
	normal force contact_lfoot.......: 442.8
	tracking err task-com............: 0.000
	||v||: 2.295	 ||dv||: 3.457
Time 1.500
	normal force contact_rfoot.......: 442.2
	normal force contact_lfoot.......: 443.5
	tracking err task-com............: 0.000
	||v||: 4.500	 ||dv||: 12.873
Time 2.000
	normal force contact_rfoot.......: 453.3
	normal force contact_lfoot.......: 431.0
	tracking err task-com............: 0.000
	||v||: 15.151	 ||dv||: 98.843
Time 2.500
	normal force contact_rfoot.......: 405.4
	normal force contact_lfoot.......: 441.9
	tracking err task-com............: 0.000
	||v||: 47.587	 ||dv||: 322.21

In [ ]:
# PLOT STUFF
time = np.arange(0.0, N_SIMULATION*dt, dt)


In [ ]:
(f, ax) = plut.create_empty_figure(3, 1, figsize=(10,10))
for i in range(3):
    ax[i].plot(time, com_pos[i,:].A1, label='CoM '+str(i))
    ax[i].plot(time, com_pos_ref[i,:].A1, 'r:', label='CoM Ref '+str(i))
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('CoM [m]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)

plt.show()

In [ ]:
(f, ax) = plut.create_empty_figure(3, 1, figsize=(10,10))
for i in range(3):
    ax[i].plot(time, com_vel[i,:].A1, label='CoM Vel '+str(i))
    ax[i].plot(time, com_vel_ref[i,:].A1, 'r:', label='CoM Vel Ref '+str(i))
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('CoM Vel [m/s]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)

plt.show()


In [ ]:

(f, ax) = plut.create_empty_figure(3, 1, figsize=(10,10))
for i in range(3):
    ax[i].plot(time, com_acc[i,:].A1, label='CoM Acc '+str(i))
    ax[i].plot(time, com_acc_ref[i,:].A1, 'r:', label='CoM Acc Ref '+str(i))
    ax[i].plot(time, com_acc_des[i,:].A1, 'g--', label='CoM Acc Des '+str(i))
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('CoM Acc [m/s^2]')
    leg = ax[i].legend()
    leg.get_frame().set_alpha(0.5)
    
plt.show()

In [ ]:
(f, ax) = plut.create_empty_figure(3, 1, figsize=(10,10))
ax[0].plot(time, err_y, label='x ')
ax[1].plot(time, err_y, label='y')
ax[2].plot(time, err_z, label='z')
ax[0].set_xlabel('Time [s]')
ax[1].set_xlabel('Time [s]')
ax[2].set_xlabel('Time [s]')
ax[0].set_ylabel('Position error x [m]')
ax[1].set_ylabel('Position error y [m]')
ax[2].set_ylabel('Position error z [m]')
    
plt.show()